In [1]:
import json
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import JSONLoader
from langchain.docstore.document import Document
from pathlib import Path
from pprint import pprint as pp
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()

file_path = "emails_100.json"

data = json.loads(Path(file_path).read_text())

documents = [
    Document(
        page_content=entry["Body"],  # Main content for the document
        metadata={
            "From": entry["From"],
            "Email": entry["Email"],
            "Subject": entry["Subject"],
            "Date": entry["Date"]
        }
    )
    for entry in data
]

if not os.path.exists("email_replier_ELI_vs"):
    os.makedirs("email_replier_ELI_vs")
    vs: FAISS = FAISS.from_documents(documents, embeddings)
    vs.save_local("email_replier_ELI_vs")
else:
    vs = FAISS.load_local("email_replier_ELI_vs", embeddings, allow_dangerous_deserialization=True)

In [7]:
for mail in vs.similarity_search("linkedin friend", k=3):
    pp(mail.metadata)
    pp(mail.page_content)
    print("-")

{'Date': '2024-11-22 06:31:09',
 'Email': 'friendsuggestion@facebookmail.com',
 'From': 'Facebook Friend Suggestions',
 'Subject': '👤 Kiran Khashempur is a new friend suggestion for you'}
('======================================== Find more friends '
 '======================================== Hi Harsh, Add the people you know '
 'to see their photos and updates. Thanks, The Facebook team '
 '======================================== This message was sent to '
 "pundhirharsh6@gmail.com. If you don't want to receive these emails from Meta "
 'in the future, please follow the link below to unsubscribe. Meta Platforms, '
 'Inc., Attention: Community Support, 1 Meta Way, Menlo Park, CA 94025 To help '
 "keep your account secure, please don't forward this email. Follow the link "
 'below to learn more.')
-
{'Date': '2024-11-24 18:57:05',
 'Email': 'notifications-noreply@linkedin.com',
 'From': 'LinkedIn',
 'Subject': 'You have 1 new invitation'}
('You have 1 new invitation View invitations: C

In [ ]:
vs.similarity_search("facebook friend", k=3, filter={'Email': 'notifications-noreply@linkedin.com'})

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint
from IPython.display import display, Markdown

retriever = vs.as_retriever(search_type="mmr", search_kwargs={"k": 3})
# docs = retriever("{input}")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that can answer questions about the emails. Use the following emails for context: {context} \
            \n answer with proper headings and bullet points",
        ),
        ("human", "{input}"),
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini")

parser = StrOutputParser()

chain = prompt | llm | parser

hum_input = "do Any emails need immediate attention?" 
# hum_input = "do any emails seek immideate attention?"

context = vs.similarity_search(hum_input, k=10)

display(
    Markdown(chain.invoke({"input": hum_input, "context": context}))
)
# print sources of the emails

display(Markdown("<ins>**Note:** The following emails were used for context:</ins>"))
for email in context:
    print(email.metadata["Email"])

### Emails That Require Immediate Attention

1. **HDFC Bank: Maintain Average Minimum Balance**
   - **Subject**: Important notice: Maintain Average Minimum Balance to avoid charges
   - **Date**: November 26, 2024
   - **Action Required**: Review your account balance to ensure it meets the minimum requirements to avoid potential charges.

2. **MijnOverheid: Document Received**
   - **Subject**: Bericht van Justis (document ontvangen)
   - **Date**: November 28, 2024
   - **Action Required**: Log in to MijnOverheid to view the document and determine if any action is needed based on its content.

### Summary of Other Emails
- **AutoGPT Newsletter**: No immediate action required.
- **Brain Food Newsletter**: No immediate action required.
- **LinkedIn Notifications**: No immediate action required unless interested in the mentioned posts or updates.
- **Facebook Friend Suggestions**: No immediate action required unless you wish to connect with suggested friends.

### Conclusion
The emails from HDFC Bank and MijnOverheid should be prioritized for immediate review and action.

<ins>**Note:** The following emails were used for context:</ins>

news@agpt.co
information@hdfcbank.net
newsletter@farnamstreetblog.com
messages-noreply@linkedin.com
noreply@alerts.sbi.co.in
messages-noreply@linkedin.com
linkedin@e.linkedin.com
messages-noreply@linkedin.com
friendsuggestion@facebookmail.com
noreply@mijn.overheid.nl


In [13]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

llm = ChatOpenAI(model="gpt-4o-mini")

syst = SystemMessage("talk like santa")
hum = HumanMessage("do any emails seek immideate attention?")

pp(llm.invoke([syst, hum]).to_json())

{'id': ['langchain', 'schema', 'messages', 'AIMessage'],
 'kwargs': {'additional_kwargs': {'refusal': None},
            'content': 'Ho ho ho! Well, my dear friend, when it comes to '
                       'emails, some certainly may require immediate '
                       'attention, just like the naughty or nice list! If you '
                       'see emails marked as urgent, have important deadlines, '
                       'or involve time-sensitive matters, those are the ones '
                       'to focus on right away. Just like checking in on your '
                       "Christmas preparations, it's always good to "
                       'prioritize! Merry emailing! 🎅🎄',
            'id': 'run-1330fffa-6ee5-4aed-8b3f-9a7171fc9fd8-0',
            'invalid_tool_calls': [],
            'response_metadata': {'finish_reason': 'stop',
                                  'logprobs': None,
                                  'model_name': 'gpt-4o-mini-2024-07-18',
          

In [14]:
from langchain_core.prompts import ChatPromptTemplate

syst = SystemMessage("talk like {name}")
hum = HumanMessage("do any emails seek immideate attention?")
prompt = ChatPromptTemplate([syst, hum])

# OR

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "talk like {name}"),
        ("human", "do any emails seek immideate attention?"),
    ]
)


prompt.invoke({"name": "santa"}).to_messages()


[SystemMessage(content='talk like santa', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='do any emails seek immideate attention?', additional_kwargs={}, response_metadata={})]

In [21]:
pp(llm.is_lc_serializable())
pp(prompt.is_lc_serializable())
pp(parser.is_lc_serializable())

True
True
True
